# Tạo Golden SQL (Câu hỏi & SQL chính xác) bằng Gemini v2

## 1. Thiết Lập và Khởi Tạo

In [16]:
import google.generativeai as genai
import pandas as pd
import os
import random
import csv
import time
from IPython.display import display, Markdown
import json

# --- Quản lý API Key ---
GEMINI_API_KEYS = [
    os.environ.get('GOOGLE_API_KEY')
]
print(os.environ.get('GOOGLE_API_KEY'))
print(f"Đã tìm thấy {len(GEMINI_API_KEYS)} API keys.")
API_RATE_LIMIT_PER_MINUTE = 100
API_KEY_COOLDOWN_SECONDS = 70
api_key_usage = {key: {"count": 0, "last_reset_time": time.time(), "blocked_until": 0} for key in GEMINI_API_KEYS}
current_api_key_index = 0

def get_next_available_api_key():
    global current_api_key_index, api_key_usage
    start_index = current_api_key_index
    while True:
        api_key_to_try = GEMINI_API_KEYS[current_api_key_index]
        usage_stats = api_key_usage[api_key_to_try]
        current_time = time.time()
        if current_time < usage_stats["blocked_until"]:
            current_api_key_index = (current_api_key_index + 1) % len(GEMINI_API_KEYS)
            if current_api_key_index == start_index:
                time.sleep(5)
            continue
        if current_time - usage_stats["last_reset_time"] > API_KEY_COOLDOWN_SECONDS:
            usage_stats["count"] = 0
            usage_stats["last_reset_time"] = current_time
        if usage_stats["count"] < API_RATE_LIMIT_PER_MINUTE:
            usage_stats["count"] += 1
            genai.configure(api_key=api_key_to_try)
            print(f"Sử dụng API Key: ...{api_key_to_try[-6:]} (Lượt: {usage_stats['count']}/{API_RATE_LIMIT_PER_MINUTE})")
            return api_key_to_try
        current_api_key_index = (current_api_key_index + 1) % len(GEMINI_API_KEYS)
        if current_api_key_index == start_index:
            print(f"Tất cả API keys đã hết lượt. Đang đợi...")
            time.sleep(10)

model = genai.GenerativeModel('gemini-2.5-pro-preview-06-05') 

NUM_QUESTIONS_TO_GENERATE = 1
OUTPUT_CSV_FILE = 'golden_sql_generated.csv'
EXISTING_QUESTIONS_FILE = 'GoldenQ.csv'
# ĐỊNH NGHĨA CỘT MỤC TIÊU
FINAL_COLUMNS = ['Category', 'Question', 'SQL', 'User_info']

None
Đã tìm thấy 1 API keys.


## 2. Tải Dữ Liệu Đầu Vào

In [17]:
try:
    try:
        df_existing_questions = pd.read_csv(EXISTING_QUESTIONS_FILE)
    except (FileNotFoundError, pd.errors.EmptyDataError):
        print(f"File {EXISTING_QUESTIONS_FILE} không tìm thấy hoặc rỗng. Sẽ tạo DataFrame mới.")
        df_existing_questions = pd.DataFrame(columns=FINAL_COLUMNS)

    # Đảm bảo tất cả các cột cần thiết đều tồn tại
    for col in FINAL_COLUMNS:
        if col not in df_existing_questions.columns:
            df_existing_questions[col] = pd.NA
            
    # Sắp xếp lại các cột theo đúng thứ tự và chỉ giữ lại những cột cần thiết
    df_existing_questions = df_existing_questions[FINAL_COLUMNS]

    print(f"Đã tải {len(df_existing_questions)} bản ghi hiện có từ {EXISTING_QUESTIONS_FILE}")

except Exception as e:
    print(f"Đã xảy ra lỗi khi tải dữ liệu hiện có: {e}")
    df_existing_questions = pd.DataFrame(columns=FINAL_COLUMNS)

required_files = {
    "permissions": 'per_ENG.txt',
    "schema": 'm-schema_final.txt',
    "student_he00001_context": 'he00001.txt',
    "lecturer_lec001_context": 'lec001.txt',
    "tm_tm001_context": 'tm001.txt'
}

file_contents = {}
for name, path in required_files.items():
    try:
        with open(path, 'r', encoding='utf-8') as f:
            file_contents[name] = f.read()
        print(f"Đã tải {name} từ {path} ({len(file_contents[name])} ký tự)")
    except FileNotFoundError:
        raise FileNotFoundError(f"Lỗi: Không tìm thấy file bắt buộc '{path}'. Vui lòng đảm bảo file tồn tại.")

permissions_text = file_contents["permissions"]
db_schema_text = file_contents["schema"]
student_he00001_context = file_contents["student_he00001_context"]
lecturer_lec001_context = file_contents["lecturer_lec001_context"]
tm_tm001_context = file_contents["tm_tm001_context"]

display(Markdown("**DB Schema Loaded (Snippet):**"))
display(Markdown(f"```sql\n{db_schema_text[:500]}...\n```"))

Đã tải 87 bản ghi hiện có từ GoldenQ.csv
Đã tải permissions từ per_ENG.txt (3687 ký tự)
Đã tải schema từ m-schema_final.txt (10518 ký tự)
Đã tải student_he00001_context từ he00001.txt (6482 ký tự)
Đã tải lecturer_lec001_context từ lec001.txt (3176 ký tự)
Đã tải tm_tm001_context từ tm001.txt (419 ký tự)


**DB Schema Loaded (Snippet):**

```sql
【DB_ID】 text_to_sql_final
【Schema】
# Table: Attendance
[
  (enrollment_id:VARCHAR(50), Primary Key, Foreign key referencing Enrollments table. Maps to Enrollments(enrollment_id). NOT NULL., Examples: ['HE00001_AI_K1_FA23_AIL301_SU24', 'HE00001_AI_K1_FA23_CSD201_SP24']),
  (schedule_id:VARCHAR(50), Primary Key, Foreign key referencing Schedules table. Maps to Schedules(schedule_id). NOT NULL., Examples: ['AI_K1_FA23_AIL301_SU24_S1', 'AI_K1_FA23_CSD201_SP24_S1']),
  (status:VARCHAR(50), Attendance...
```

## 3. Các Hàm Hỗ Trợ

In [18]:
FIXED_USERS = [
    {"id": "HE00001", "role": "Student", "context": student_he00001_context},
    {"id": "LEC001", "role": "Lecturer", "context": lecturer_lec001_context},
    {"id": "TM001", "role": "Training Manager", "context": tm_tm001_context}
]
USER_WEIGHTS = [2, 1, 7]

def get_random_weighted_user_info():
    chosen_user = random.choices(FIXED_USERS, weights=USER_WEIGHTS, k=1)[0]
    user_info_str = f"Role: {chosen_user['role']}, ID: {chosen_user['id']}"
    return user_info_str, chosen_user['role'], chosen_user['context']

def get_relevant_permissions(full_permissions_text, user_role):
    lines = full_permissions_text.split('\n')
    relevant_lines = []
    in_role_section = False
    for line in lines:
        line_strip_lower = line.strip().lower()
        if f"{user_role.lower()}" in line_strip_lower and 'common' not in line_strip_lower:
            in_role_section = True
            continue
        if in_role_section:
            if line.strip().lower() in ['student', 'lecturer', 'training manager']:
                break
            if line.strip():
                relevant_lines.append(line.strip())
    return "\n".join(relevant_lines) if relevant_lines else f"Full system access or permissions are defined implicitly."

def get_existing_examples(df_existing, num_examples=3):
    if df_existing.empty or len(df_existing) < num_examples:
        return "(No examples available)"
    # Lấy ví dụ từ các cột Question và SQL, bỏ qua các cột khác
    sample_df = df_existing.sample(min(num_examples, len(df_existing)))
    examples_str = ""
    for _, row in sample_df.iterrows():
        question = str(row.get('Question', 'N/A'))
        sql = str(row.get('SQL', 'N/A')).replace('\n', ' ')
        examples_str += f"- Question: \"{question}\" | SQL: \"{sql}\"\n"
    return examples_str.strip()

# SỬA LỖI: Hàm này tìm các key 'question' và 'sql' viết thường từ output của Gemini
def clean_and_parse_gemini_output(raw_text):
    cleaned_text = raw_text.strip()
    if cleaned_text.startswith("```json"):
        cleaned_text = cleaned_text[len("```json"):].strip()
        if cleaned_text.endswith("```"):
            cleaned_text = cleaned_text[:-len("```")].strip()
    elif cleaned_text.startswith("```"):
        cleaned_text = cleaned_text[len("```"):].strip()
        if cleaned_text.endswith("```"):
            cleaned_text = cleaned_text[:-len("```")].strip()

    try:
        parsed_json = json.loads(cleaned_text)
        if isinstance(parsed_json, dict) and 'question' in parsed_json and 'sql' in parsed_json:
            return parsed_json
        else:
            print(f"Cảnh báo: JSON không đúng định dạng. Thiếu key 'question' hoặc 'sql'. Output: {cleaned_text[:150]}")
            return None
    except json.JSONDecodeError:
        print(f"Cảnh báo: Không thể phân tích JSON. Output: {cleaned_text[:150]}...")
        return None

## 4. Hàm Tạo Prompt Chính cho Gemini

In [19]:
def create_golden_sql_prompt(user_info_str, user_role, detailed_user_context, permissions_str, db_schema_str, examples_str):
    # user_info_str ở đây chỉ dùng để hiển thị trong prompt, không phải là cột cuối cùng
    prompt_user_profile = f"- Role: {user_role}\\n- ID: {user_info_str.split('ID: ')[-1]}"

    # Ví dụ về định dạng SQL mong muốn
    sql_format_example = """SELECT
    Cou.course_name,
    CC.class_id,
    Sch.start_time,
    Sch.end_time,
    Sch.room
FROM Schedules AS Sch
JOIN ClassCourse AS CC
    ON Sch.class_course_id = CC.class_course_id
JOIN Courses AS Cou
    ON CC.course_id = Cou.course_id
WHERE
    CC.lecturer_id = 'LEC001'"""

    mssql_specific_guidance = (
        "When appropriate, use Microsoft SQL Server (T-SQL) specific functions and keywords. "
        "For example: use `TOP 10` instead of `LIMIT 10`, use `GETDATE()` for the current timestamp, "
        "use `STRING_AGG()` for aggregation, or `DATEPART()` for date manipulation."
    )

    prompt = f"""
You are an expert database administrator (DBA) specializing in Microsoft SQL Server.
Your primary task is to generate a valid, realistic natural language question and its corresponding correct, executable SQL query.

**Instructions:**
1.  **Analyze the User:** Carefully review the user's profile, their detailed context, and their specific permissions.
2.  **Create a Permissible Question:** Generate a new, unique natural language `question` that this specific user might ask. The question **MUST** be something the user is **ALLOWED** to ask according to their permissions. **Crucially, the question should be concise and to the point (ideally 1-2 sentences), reflecting how a real person asks.**
3.  **Generate Correct MS SQL (T-SQL):** Write the corresponding `sql` query using **Microsoft SQL Server (T-SQL) syntax**. The query must be syntactically correct and **MUST be formatted for readability** with newlines and indentation.
4.  **Adhere to Context:** The question and SQL must be logically consistent with the user's detailed information (e.g., if the user is HE00001, the SQL should use `WHERE student_id = 'HE00001'`).
5.  **Output Format:** Your final output **MUST** be a single, clean JSON object with two keys: "question" and "sql". Do not add any other text or formatting.

--- CONTEXT ---

**1. User Profile:**
{prompt_user_profile}

**2. User's Detailed Information (Primary Context):**
```
{detailed_user_context}
```

**3. User's Permissions (Rules for what is ALLOWED):**
{permissions_str}

**4. Database (DB) Schema:**
```sql
{db_schema_str}
```

**5. Example of DESIRED SQL FORMAT (Your SQL output MUST follow this style):
{sql_format_example}

**6. Existing Examples (for style inspiration, DO NOT COPY):**
{examples_str}

**7. Dialect-Specific Guidance:
{mssql_specific_guidance}

--- END CONTEXT ---

Now, generate the JSON object containing the new, unique, and permissible question and its correct SQL query.

JSON Output:
"""
    return prompt

## 5. Vòng Lặp Tạo Dữ Liệu

In [20]:
generated_data = []
successful_generations = 0
total_attempts = 0
max_total_attempts = NUM_QUESTIONS_TO_GENERATE * 5 # Tối đa 5 lần thử cho mỗi câu hỏi

existing_questions_set = set(df_existing_questions['Question'].str.lower().dropna())

print(f"Bắt đầu quá trình tạo {NUM_QUESTIONS_TO_GENERATE} cặp câu hỏi/SQL...")

while successful_generations < NUM_QUESTIONS_TO_GENERATE and total_attempts < max_total_attempts:
    total_attempts += 1
    print(f"\n--- Đang thử tạo cặp {successful_generations + 1}/{NUM_QUESTIONS_TO_GENERATE} (Lần thử: {total_attempts}) ---")

    api_key = get_next_available_api_key()
    if not api_key:
        print("Không có API key khả dụng. Dừng lại.")
        break
    
    # THAY ĐỔI: Lấy cả user_info_str và user_role để lưu lại
    user_info_str, user_role, detailed_context = get_random_weighted_user_info()
    permissions_str = get_relevant_permissions(permissions_text, user_role)
    examples_str = get_existing_examples(df_existing_questions)
    prompt = create_golden_sql_prompt(user_info_str, user_role, detailed_context, permissions_str, db_schema_text, examples_str)

    try:
        print(f"Đang gọi API Gemini cho user {user_role}...")
        safety_settings = [{"category": c, "threshold": "BLOCK_NONE"} for c in
                         ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH",
                          "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]]
        generation_config = genai.types.GenerationConfig(candidate_count=1, temperature=0.5)
        response = model.generate_content(prompt, safety_settings=safety_settings, generation_config=generation_config)

        if response.parts:
            parsed_output = clean_and_parse_gemini_output(response.text)
            if parsed_output:
                new_question = parsed_output['question']
                new_sql = parsed_output['sql']

                if new_question.lower() not in existing_questions_set:
                    print(f"Thành công! Câu hỏi: {new_question}")
                    
                    # THAY ĐỔI: Thêm dữ liệu vào dict với 4 cột
                    new_row = {
                        'Category': user_role,
                        'Question': new_question,
                        'SQL': new_sql,
                        'User_info': user_info_str
                    }
                    generated_data.append(new_row)
                    existing_questions_set.add(new_question.lower())
                    successful_generations += 1
                else:
                    print(f"Câu hỏi bị trùng lặp, bỏ qua: {new_question}")
            else:
                print("Tạo cặp câu hỏi/SQL thất bại do output không hợp lệ.")
        else:
            print("Phản hồi API Gemini rỗng hoặc bị chặn.")

    except Exception as e:
        print(f"Lỗi khi gọi API Gemini: {e}")

    time.sleep(random.uniform(2.0, 4.0))

print(f"\nHoàn thành: Đã tạo thành công {successful_generations} bản ghi mới.")

Bắt đầu quá trình tạo 1 cặp câu hỏi/SQL...

--- Đang thử tạo cặp 1/1 (Lần thử: 1) ---


TypeError: 'NoneType' object is not subscriptable

## 6. Lưu Kết Quả

In [ ]:
if generated_data:
    # Chuyển list dữ liệu mới tạo thành DataFrame
    df_new_questions = pd.DataFrame(generated_data, columns=FINAL_COLUMNS)

    # (Tùy chọn nhưng nên có) Đảm bảo không có dòng nào trống hoặc trùng lặp trong chính lần tạo này
    df_new_questions.drop_duplicates(subset=['Question'], keep='last', inplace=True)
    df_new_questions.dropna(subset=['Question'], inplace=True)
    df_new_questions = df_new_questions[df_new_questions['Question'] != '']

    # Chỉ lưu các bản ghi MỚI vào file đầu ra
    df_new_questions.to_csv(OUTPUT_CSV_FILE, index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

    print(f"\nĐã lưu thành công {len(df_new_questions)} bản ghi MỚI vào file {OUTPUT_CSV_FILE}")
    print("Hiển thị các bản ghi vừa được tạo:")
    display(df_new_questions)
else:
    print("\nKhông có dữ liệu mới nào được tạo trong lần chạy này.")


Đã lưu thành công 50 bản ghi MỚI vào file golden_sql_generated.csv
Hiển thị các bản ghi vừa được tạo:


,Category,Question,SQL,User_info
0,Student,How many times was I absent in the Programming...,SELECT\n COUNT(T1.status)\nFROM Attendance AS...,"Role: Student, ID: HE00001"
1,Training Manager,Can you provide a breakdown of student enrollm...,"SELECT\n M.major_name,\n COUNT(S.student_id)...","Role: Training Manager, ID: TM001"
2,Lecturer,Which of my students scored higher than an 8.0...,"SELECT\n U.fullname,\n C.course_name,\n E.a...","Role: Lecturer, ID: LEC001"
3,Student,How many times was I absent from my Programmin...,SELECT\n COUNT(T1.status)\nFROM Attendance AS...,"Role: Student, ID: HE00001"
4,Student,What were my scores for each graded component ...,"SELECT\n T4.grade_name,\n T1.grade_value...","Role: Student, ID: HE00001"
5,Training Manager,What is the total number of students enrolled ...,"SELECT\n M.major_name,\n COUNT(S.student...","Role: Training Manager, ID: TM001"
6,Training Manager,Can you show me the top 5 courses with the hig...,"SELECT\n C.course_name,\n AVG(E.average) AS ...","Role: Training Manager, ID: TM001"
7,Student,What are my detailed grades for the Artificial...,"SELECT\n T3.grade_name,\n T3.grade_weight,\n...","Role: Student, ID: HE00001"
8,Training Manager,Which lecturers are teaching more than two cou...,"SELECT\n U.fullname,\n COUNT(CC.course_id) A...","Role: Training Manager, ID: TM001"
9,Training Manager,Can you give me a count of courses assigned to...,"SELECT\n U.fullname,\n COUNT(CC.course_i...","Role: Training Manager, ID: TM001"
